<a href="https://colab.research.google.com/github/linzhe001/tutorial_notebooks/blob/main/LLM_GPT2_LLaMA_QA_LoRA_Working_All.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12

GPT2_FFT

In [2]:
import torch
import os
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from datasets import Dataset
import torch.nn.functional as F
import random
import numpy as np

# Seed setting function
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Set the seed for reproducibility
seed = 50
set_seed(seed)

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare 5 QA samples for training and validation
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What is the largest planet?", "answer": "The largest planet is Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "What is the speed of light?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Can you tell me who authored '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What planet is the biggest in our solar system?", "answer": "The largest planet is Jupiter."},
    {"question": "Who is the artist behind the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "How fast does light travel?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=60)

    # Clone input_ids into labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = tokenizer.eos_token_id  # Ignore question and padding tokens

    inputs["labels"] = labels
    return inputs

# Convert samples to dataset and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# Convert to PyTorch DataLoader
batch_size = 2

def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch])
    attention_mask = torch.tensor([item["attention_mask"] for item in batch])
    labels = torch.tensor([item["labels"] for item in batch])
    return input_ids, attention_mask, labels

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Cross-entropy loss function ignoring padding tokens
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Function to save the best model based on validation loss
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./gpt2-qa-best-loss-cml"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training and evaluation functions
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=10):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            # Forward pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift labels and logits for proper alignment
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            # Flatten logits and labels for loss calculation
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute loss
            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")

        # Save best model based on validation loss
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss)

def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift labels and logits for proper alignment
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()

            # Flatten logits and labels for loss calculation
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute loss
            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Start training
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=10)

# Load the best fine-tuned model for inference
model_name = "./gpt2-qa-best-loss-cml"
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Inference function
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Training Loss: 4.4203, Validation Loss: 1.5726
Best model saved at epoch 1 with validation loss: 1.5726
Epoch 2/10, Training Loss: 0.9883, Validation Loss: 0.9799
Best model saved at epoch 2 with validation loss: 0.9799
Epoch 3/10, Training Loss: 0.7012, Validation Loss: 0.6840
Best model saved at epoch 3 with validation loss: 0.6840
Epoch 4/10, Training Loss: 0.4719, Validation Loss: 0.4885
Best model saved at epoch 4 with validation loss: 0.4885
Epoch 5/10, Training Loss: 0.2782, Validation Loss: 0.4139
Best model saved at epoch 5 with validation loss: 0.4139
Epoch 6/10, Training Loss: 0.2091, Validation Loss: 0.3706
Best model saved at epoch 6 with validation loss: 0.3706
Epoch 7/10, Training Loss: 0.1939, Validation Loss: 0.3273
Best model saved at epoch 7 with validation loss: 0.3273
Epoch 8/10, Training Loss: 0.1228, Validation Loss: 0.2700
Best model saved at epoch 8 with validation loss: 0.2700
Epoch 9/10, Training Loss: 0.1057, Validation Loss: 0.2148
Best model sa

GPT2_LoRA_HF_Trainer

In [9]:
import torch
import os
import random
import numpy as np
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TrainingArguments, Trainer
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType, PeftModelForCausalLM, PeftModel


# Set seed for reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed = 50
set_seed(seed)

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Configure LoRA adapters
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_proj"]
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Verify trainable LoRA parameters

# Move model to device
model.to(device)

# Prepare QA samples
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What is the largest planet?", "answer": "The largest planet is Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "What is the speed of light?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Can you tell me who authored '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What planet is the biggest in our solar system?", "answer": "The largest planet is Jupiter."},
    {"question": "Who is the artist behind the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "How fast does light travel?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=60)

    # Clone input_ids into labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = -100  # Ignore question and padding tokens

    inputs["labels"] = labels  # Ensure labels are correctly set
    return inputs

# Convert samples to dataset and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# Modified collate function for Trainer API compatibility
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch], dtype=torch.long)
    attention_mask = torch.tensor([item["attention_mask"] for item in batch], dtype=torch.long)
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.long)
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

# Training Arguments
training_args = TrainingArguments(
    output_dir="./gpt2-lora-qa",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=20,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to=[],
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid,
    tokenizer=tokenizer,
    data_collator=collate_fn  # Ensure correct batch formatting
)

# Start training
trainer.train()

# Save LoRA fine-tuned model
trainer.save_model("./gpt2-lora-qa")


# Load the base GPT-2 model first
base_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

# Load the LoRA adapter and merge it with the base model
model = PeftModel.from_pretrained(base_model, "./gpt2-lora-qa").to(device)

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding


# Inference function
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(device)
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)
    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-1d037cfc3a43>:110: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Q: What is the capital of France?
A: The capital of France is Paris.

Question


GPT2_LoRA_From_Scratch

In [15]:
import torch
import os
import random
import numpy as np
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType

# Set seed for reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(50)

# Load tokenizer and GPT-2 model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

base_model = GPT2LMHeadModel.from_pretrained(model_name)
base_model.config.pad_token_id = tokenizer.eos_token_id
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model.to(device)

# LoRA Configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_proj"]
)

# Wrap model with LoRA
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# QA Samples for Training & Validation
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What is the largest planet?", "answer": "The largest planet is Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "What is the speed of light?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Can you tell me who authored '1984'?", "answer": "George Orwell wrote '1984'."},
    {"question": "What planet is the biggest in our solar system?", "answer": "The largest planet is Jupiter."},
    {"question": "Who is the artist behind the Mona Lisa?", "answer": "Leonardo da Vinci painted the Mona Lisa."},
    {"question": "How fast does light travel?", "answer": "The speed of light is approximately 299,792 kilometers per second."}
]

# Data Preprocessing Function
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=60)

    # Clone input_ids into labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = -100  # Ignore question and padding tokens

    inputs["labels"] = labels
    return inputs

# Convert to Dataset and Preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# DataLoader Collation Function
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch], dtype=torch.long)
    attention_mask = torch.tensor([item["attention_mask"] for item in batch], dtype=torch.long)
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.long)
    return input_ids, attention_mask, labels

# DataLoader Creation
batch_size = 2
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer & Loss Function
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Model Saving Function
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./gpt2-lora-qa-best"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"✅ Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training Function
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=5):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift logits and labels for loss computation
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute Loss
            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Save best model
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss)

# Validation Function
def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Start Training
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=20)

from peft import PeftModel
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Define the path where the best model is saved
best_model_path = "./gpt2-lora-qa-best"

# Load the base GPT-2 model first
base_model = GPT2LMHeadModel.from_pretrained("gpt2").to("cuda" if torch.cuda.is_available() else "cpu")

# Load the LoRA adapter and merge it with the base model
model = PeftModel.from_pretrained(base_model, best_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

# Load the trained tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(best_model_path)
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding

# Function to generate answers using the fine-tuned model
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question} Answer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=60).to(model.device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")

trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Epoch 1/20, Train Loss: 1.5768, Val Loss: 1.5593
✅ Best model saved at epoch 1 with validation loss: 1.5593
Epoch 2/20, Train Loss: 1.4712, Val Loss: 1.5486
✅ Best model saved at epoch 2 with validation loss: 1.5486
Epoch 3/20, Train Loss: 1.4281, Val Loss: 1.5368
✅ Best model saved at epoch 3 with validation loss: 1.5368
Epoch 4/20, Train Loss: 1.7802, Val Loss: 1.5240
✅ Best model saved at epoch 4 with validation loss: 1.5240
Epoch 5/20, Train Loss: 1.4239, Val Loss: 1.5100
✅ Best model saved at epoch 5 with validation loss: 1.5100
Epoch 6/20, Train Loss: 1.7470, Val Loss: 1.4947
✅ Best model saved at epoch 6 with validation loss: 1.4947
Epoch 7/20, Train Loss: 1.4601, Val Loss: 1.4768
✅ Best model saved at epoch 7 with validation loss: 1.4768
Epoch 8/20, Train Loss: 1.4138, Val Loss: 1.4569
✅ Best model saved at epoch 8 with validation loss: 1.4569
Epoch 9/20, Train Loss: 1.5599, Val Loss: 1.4359
✅ Best model saved at epoch 9 with validation loss: 1.4359
Epoch 10/20, Train Loss: 1.5

#Attempt on LLaMA

In [2]:
!pip -q install sentencepiece
!pip -q install transformers peft bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.6 MB/s eta 0:00:00


In [19]:
from huggingface_hub import login
# Replace 'your_huggingface_token' with your actual token
HUGGINGFACE_TOKEN = "your token"

# Authenticate with Hugging Face
login(HUGGINGFACE_TOKEN)

print("✅ Successfully logged into Hugging Face!")

✅ Successfully logged into Hugging Face!


Without quantization:

In [1]:
import torch
import os
import random
import numpy as np
from torch.utils.data import DataLoader
from transformers import LlamaForCausalLM, LlamaTokenizer, AdamW
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoTokenizer

# Set random seed for reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load LLaMA 2.7B tokenizer and model
model_name = "Meta-Llama/Llama-2-7b-chat-hf"
# tokenizer = LlamaTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token  # Set padding token

# Load the correct tokenizer for LLaMA-2 using AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_fast=False)

# Set padding token to EOS if missing
tokenizer.pad_token = tokenizer.eos_token

base_model = LlamaForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map="auto"
)
base_model.config.pad_token_id = tokenizer.eos_token_id  # Set pad token ID

# Configure LoRA for memory-efficient fine-tuning
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # Apply LoRA to attention layers
)

# Wrap the model with LoRA adapters
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()  # Verify LoRA trainable parameters

# Define QA training and validation samples
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell."},
    {"question": "What is the largest planet?", "answer": "Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "What is the speed of light?", "answer": "299,792 km/s."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "Paris."},
    {"question": "Who is the author of '1984'?", "answer": "George Orwell."},
    {"question": "What planet is the biggest?", "answer": "Jupiter."},
    {"question": "Who created the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "How fast does light travel?", "answer": "299,792 km/s."}
]

# Preprocessing function
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=256)

    # Copy input_ids to labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = -100  # Ignore these tokens in loss computation

    inputs["labels"] = labels
    return inputs

# Convert datasets and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# DataLoader Collation
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch], dtype=torch.long)
    attention_mask = torch.tensor([item["attention_mask"] for item in batch], dtype=torch.long)
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.long)
    return input_ids, attention_mask, labels

# Create DataLoaders
batch_size = 2
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer & Loss Function
optimizer = AdamW(model.parameters(), lr=3e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Function to save the best model
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./llama-lora-qa-best"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"✅ Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training Function
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=3):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift logits and labels for loss computation
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute Loss
            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Save best model
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss)

# Validation Function
def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Start Training
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=3)


import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel

# Path to the best fine-tuned model
best_model_path = "./llama-lora-qa-best"

# Load the base LLaMA 2.7B model
base_model = LlamaForCausalLM.from_pretrained(
    "Meta-Llama/Llama-2-7b-chat-hf",
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load the LoRA fine-tuned adapter
model = PeftModel.from_pretrained(base_model, best_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned tokenizer
tokenizer = LlamaTokenizer.from_pretrained(best_model_path)
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding

# Function to generate answers
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(model.device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.1243


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 16.12 MiB is free. Process 182321 has 14.72 GiB memory in use. Of the allocated memory 14.53 GiB is allocated by PyTorch, and 70.41 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

With quantization

In [3]:
import torch
import os
import random
import numpy as np
from torch.utils.data import DataLoader
from transformers import LlamaForCausalLM, LlamaTokenizer, AdamW
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoTokenizer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType

# Set random seed for reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load LLaMA 2.7B tokenizer and model
model_name = "Meta-Llama/Llama-2-7b-chat-hf"
# tokenizer = LlamaTokenizer.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token  # Set padding token

# Load the correct tokenizer for LLaMA-2 using AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Set padding token to EOS if missing
tokenizer.pad_token = tokenizer.eos_token

# base_model = LlamaForCausalLM.from_pretrained(
#     model_name, torch_dtype=torch.float16, device_map="auto"
# )

# Configure 4-bit quantization using bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # Normalized Float 4 (better than standard FP4)
    bnb_4bit_use_double_quant=True,  # Uses secondary quantization for better precision
    bnb_4bit_compute_dtype=torch.float16  # Keeps computation in FP16 for stability
)

# uncomment these first time
# Load LLaMA 2.7B with 4-bit quantization
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

base_model.config.pad_token_id = tokenizer.eos_token_id  # Set pad token ID

# Configure LoRA for memory-efficient fine-tuning
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # Apply LoRA to attention layers
)

# Wrap the model with LoRA adapters
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()  # Verify LoRA trainable parameters

# Define QA training and validation samples
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell."},
    {"question": "What is the largest planet?", "answer": "Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "What is the speed of light?", "answer": "299,792 km/s."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "Paris."},
    {"question": "Who is the author of '1984'?", "answer": "George Orwell."},
    {"question": "What planet is the biggest?", "answer": "Jupiter."},
    {"question": "Who created the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "How fast does light travel?", "answer": "299,792 km/s."}
]

# Preprocessing function
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=256)

    # Copy input_ids to labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = -100  # Ignore these tokens in loss computation

    inputs["labels"] = labels
    return inputs

# Convert datasets and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# DataLoader Collation
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch], dtype=torch.long)
    attention_mask = torch.tensor([item["attention_mask"] for item in batch], dtype=torch.long)
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.long)
    return input_ids, attention_mask, labels

# Create DataLoaders
batch_size = 2
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer & Loss Function
optimizer = AdamW(model.parameters(), lr=3e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Function to save the best model
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./llama-lora-qa-best"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"✅ Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training Function
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=3):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift logits and labels for loss computation
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute Loss
            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Save best model
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss)

# Validation Function
def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Start Training
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=20)


import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel

# Path to the best fine-tuned model
best_model_path = "./llama-lora-qa-best"

# Load the base LLaMA 2.7B model
# base_model = LlamaForCausalLM.from_pretrained(
#     "Meta-Llama/Llama-2-7b-chat-hf",
#     torch_dtype=torch.float16,
#     device_map="auto"
# )

# base_model = AutoModelForCausalLM.from_pretrained(
#     "meta-llama/Llama-2-7b-chat-hf",
#     quantization_config=bnb_config,
#     device_map="auto"
# )

# Load the LoRA fine-tuned adapter
model = PeftModel.from_pretrained(base_model, best_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned tokenizer
tokenizer = LlamaTokenizer.from_pretrained(best_model_path)
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding

# Function to generate answers
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(model.device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")

trainable params: 8,388,608 || all params: 6,746,804,224 || trainable%: 0.1243


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Epoch 1/20, Train Loss: 1.3395, Val Loss: 1.1618
✅ Best model saved at epoch 1 with validation loss: 1.1618
Epoch 2/20, Train Loss: 0.9050, Val Loss: 0.9045
✅ Best model saved at epoch 2 with validation loss: 0.9045
Epoch 3/20, Train Loss: 0.9064, Val Loss: 0.6462
✅ Best model saved at epoch 3 with validation loss: 0.6462
Epoch 4/20, Train Loss: 0.5318, Val Loss: 0.4623
✅ Best model saved at epoch 4 with validation loss: 0.4623
Epoch 5/20, Train Loss: 0.2958, Val Loss: 0.3614
✅ Best model saved at epoch 5 with validation loss: 0.3614
Epoch 6/20, Train Loss: 0.2159, Val Loss: 0.2995
✅ Best model saved at epoch 6 with validation loss: 0.2995
Epoch 7/20, Train Loss: 0.2165, Val Loss: 0.2550
✅ Best model saved at epoch 7 with validation loss: 0.2550
Epoch 8/20, Train Loss: 0.1603, Val Loss: 0.2218
✅ Best model saved at epoch 8 with validation loss: 0.2218
Epoch 9/20, Train Loss: 0.1552, Val Loss: 0.1919
✅ Best model saved at epoch 9 with validation loss: 0.1919
Epoch 10/20, Train Loss: 0.1

In [7]:
# Function to generate answers
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(model.device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "Provide only short answer of the question: Who created the Mona Lisa?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")

Q: Provide only short answer of the question: Who created the Mona Lisa?
A: Leonardo da Vinci.


LLaMA 3

In [13]:
import torch
import os
import random
import numpy as np
from torch.utils.data import DataLoader
from transformers import LlamaForCausalLM, LlamaTokenizer, AdamW
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
from transformers import AutoTokenizer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType

# Set random seed for reproducibility
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(50)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load LLaMA 2.7B tokenizer and model
# model_name = "Meta-Llama/Llama-2-7b-chat-hf"
model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
# tokenizer = LlamaTokenizer.from_pretrained(model_name)

# Load the correct tokenizer for LLaMA-2 using AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Set padding token to EOS if missing
tokenizer.pad_token = tokenizer.eos_token

# base_model = LlamaForCausalLM.from_pretrained(
#     model_name, torch_dtype=torch.float16, device_map="auto"
# )

# Configure 4-bit quantization using bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # Normalized Float 4 (better than standard FP4)
    bnb_4bit_use_double_quant=True,  # Uses secondary quantization for better precision
    bnb_4bit_compute_dtype=torch.float16  # Keeps computation in FP16 for stability
)

# Load LLaMA 2.7B with 4-bit quantization
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

base_model.config.pad_token_id = tokenizer.eos_token_id  # Set pad token ID

# Configure LoRA for memory-efficient fine-tuning
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # Apply LoRA to attention layers
)

# Wrap the model with LoRA adapters
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()  # Verify LoRA trainable parameters

# Define QA training and validation samples
train_qa_samples = [
    {"question": "What is the capital of France?", "answer": "Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell."},
    {"question": "What is the largest planet?", "answer": "Jupiter."},
    {"question": "Who painted the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "What is the speed of light?", "answer": "299,792 km/s."}
]

valid_qa_samples = [
    {"question": "Which city is the capital of France?", "answer": "Paris."},
    {"question": "Who is the author of '1984'?", "answer": "George Orwell."},
    {"question": "What planet is the biggest?", "answer": "Jupiter."},
    {"question": "Who created the Mona Lisa?", "answer": "Leonardo da Vinci."},
    {"question": "How fast does light travel?", "answer": "299,792 km/s."}
]

# Preprocessing function
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=256)

    # Copy input_ids to labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = -100  # Ignore these tokens in loss computation

    inputs["labels"] = labels
    return inputs

# Convert datasets and preprocess
dataset_train = Dataset.from_list(train_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])
dataset_valid = Dataset.from_list(valid_qa_samples).map(preprocess_data, remove_columns=["question", "answer"])

# DataLoader Collation
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch], dtype=torch.long)
    attention_mask = torch.tensor([item["attention_mask"] for item in batch], dtype=torch.long)
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.long)
    return input_ids, attention_mask, labels

# Create DataLoaders
batch_size = 2
train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(dataset_valid, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Optimizer & Loss Function
optimizer = AdamW(model.parameters(), lr=3e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Function to save the best model
def save_best_model(model, tokenizer, epoch, best_loss, current_loss, save_path="./llama-lora-qa-best"):
    if current_loss < best_loss:
        best_loss = current_loss
        os.makedirs(save_path, exist_ok=True)
        model.save_pretrained(save_path)
        tokenizer.save_pretrained(save_path)
        print(f"✅ Best model saved at epoch {epoch} with validation loss: {best_loss:.4f}")
    return best_loss

# Training Function
def train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=3):
    best_val_loss = float("inf")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0

        for batch in train_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            optimizer.zero_grad()

            # Forward Pass
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Shift logits and labels for loss computation
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            # Compute Loss
            loss = criterion(shift_logits, shift_labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        avg_val_loss = validate(model, valid_loader, criterion)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

        # Save best model
        best_val_loss = save_best_model(model, tokenizer, epoch + 1, best_val_loss, avg_val_loss)

# Validation Function
def validate(model, dataloader, criterion):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))
            shift_labels = shift_labels.view(-1)

            loss = criterion(shift_logits, shift_labels)
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Start Training
train(model, train_loader, valid_loader, optimizer, criterion, num_epochs=20)


import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel

# Path to the best fine-tuned model
best_model_path = "./llama-lora-qa-best"

# Load the base LLaMA 2.7B model
# base_model = LlamaForCausalLM.from_pretrained(
#     "Meta-Llama/Llama-2-7b-chat-hf",
#     torch_dtype=torch.float16,
#     device_map="auto"
# )

# base_model = AutoModelForCausalLM.from_pretrained(
#     "meta-llama/Llama-2-7b-chat-hf",
#     quantization_config=bnb_config,
#     device_map="auto"
# )

# Load the LoRA fine-tuned adapter
model = PeftModel.from_pretrained(base_model, best_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

# Load the fine-tuned tokenizer
# tokenizer = LlamaTokenizer.from_pretrained(best_model_path)
tokenizer = AutoTokenizer.from_pretrained(best_model_path, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token  # Ensure correct padding

# Function to generate answers
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(model.device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "What is the capital of France?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")

trainable params: 1,703,936 || all params: 1,237,518,336 || trainable%: 0.1377


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Epoch 1/20, Train Loss: 0.8744, Val Loss: 0.9499
✅ Best model saved at epoch 1 with validation loss: 0.9499
Epoch 2/20, Train Loss: 0.8475, Val Loss: 0.8506
✅ Best model saved at epoch 2 with validation loss: 0.8506
Epoch 3/20, Train Loss: 0.6850, Val Loss: 0.7530
✅ Best model saved at epoch 3 with validation loss: 0.7530
Epoch 4/20, Train Loss: 0.4630, Val Loss: 0.6541
✅ Best model saved at epoch 4 with validation loss: 0.6541
Epoch 5/20, Train Loss: 0.3314, Val Loss: 0.5708
✅ Best model saved at epoch 5 with validation loss: 0.5708
Epoch 6/20, Train Loss: 0.3336, Val Loss: 0.4996
✅ Best model saved at epoch 6 with validation loss: 0.4996
Epoch 7/20, Train Loss: 0.2611, Val Loss: 0.4373
✅ Best model saved at epoch 7 with validation loss: 0.4373
Epoch 8/20, Train Loss: 0.1585, Val Loss: 0.3819
✅ Best model saved at epoch 8 with validation loss: 0.3819
Epoch 9/20, Train Loss: 0.1165, Val Loss: 0.3395
✅ Best model saved at epoch 9 with validation loss: 0.3395
Epoch 10/20, Train Loss: 0.1

In [12]:
# Function to generate answers
def generate_answer(question):
    model.eval()
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=256).to(model.device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

    answer = tokenizer.decode(output[0], skip_special_tokens=True).split("Answer:")[-1].strip()
    return answer

# Example inference
question = "Provide only short answer of the question: Who created the Mona Lisa?"
answer = generate_answer(question)
print(f"Q: {question}\nA: {answer}")

Q: Provide only short answer of the question: Who created the Mona Lisa?
A: Leonardo da Vinci.


In [14]:
import transformers

print(transformers.__version__)

4.47.1


In [15]:
import bitsandbytes

print(bitsandbytes.__version__)

0.45.1
